# Coastal flood hazard for displacement risk modelling

This tutorial shows how to use the coastal flood hazard return periods maps.

Technical specifications for the flood maps.

**Flood map:**  
Resolution: 1000m x 1000m (approximately)  
Horizontal coordinate System: EPSG:4326 - WGS84.  
Nodata value: None.  

The data is single-banded, where each pixel value denotes the depth of the water in meters.  
*** NOTE: Best way to visualize is to clip the raster values from 0.01m to 100m. This will remove 0 values.

**Representative concentration pathways (RCP) considered for sea level rise:** RCP2.6, RCP4.5, RCP8.5  
**Future years considered for sea level rise:** 2020, 2050, 2100.  
**Return periods considered for flood maps:** 1,10,25,50,100,250,1000.   

The superfolder specifies the name of our DEM, namely venDEM. venDEM_scaled_1km implies that flood computation was done on finer (30m resolution), after which maps were rescaled to 1km (approximately).  
The set of flood maps are partitioned into unit latitude-longitude square tiles inside the superfolder.  
For example, S02E040 represents the latitude-longitude tile with bottom left corner at latitude -2 (2S) and longitude 40 (40E).

In each folder, there is then a combination of RCP and year, and within each RCP_year folder, there is one tif file per return period for the latitude-longitude tile.

In [1]:
import os
import pandas as pd
import numpy as np

os.chdir('/Users/simonameiler/Documents/WCR/Displacement/global-displacement-risk')
#os.chdir('/cluster/project/climate/evelynm/global-displacement-risk') # change back to root folder, not "~/doc"
import coastal_flood_hazard, exposure, vulnerability

Select latitude-longitude tiles for country of interest only.

First, load exposure and get lat/lon max/mins from it.  
Then load the respective flood tiles.

## Load exposure from BEM

In [2]:
from climada.entity.exposures import Exposures

In [3]:
cntry_name = 'Taiwan'

In [4]:
# Load the full dataframe, without further re-aggregation / processing other than adding centroids
gdf_bem_subcomps = exposure.gdf_from_bem_subcomps(cntry_name, opt='full')
gdf_bem_subcomps.head()

,id_1x,iso3,cpx,sector,se_seismo,valhum,valfis,bd_1_floor,bd_2_floor,bd_3_floor,geometry
0,157348471,TWN,3,beds_priv,W,0.003311,0.000035,51.785,48.215,0.0,POINT (118.42083 24.47917)
1,157348471,TWN,3,beds_priv,UFB,0.006622,0.000070,51.785,48.215,0.0,POINT (118.42083 24.47917)
2,157348471,TWN,3,beds_priv,S4M,0.005519,0.000059,51.785,48.215,0.0,POINT (118.42083 24.47917)
3,157348471,TWN,3,beds_priv,S4L,0.001656,0.000018,51.785,48.215,0.0,POINT (118.42083 24.47917)
4,157348471,TWN,3,beds_priv,S4H,0.014900,0.000158,51.785,48.215,0.0,POINT (118.42083 24.47917)


In [5]:
# filter and apply impf id
gdf_bem_subcomps = gdf_bem_subcomps[gdf_bem_subcomps['valhum']>1] # filter out rows with basically no population
gdf_bem_subcomps['impf_FL'] = gdf_bem_subcomps.apply(lambda row: vulnerability.DICT_PAGER_FLIMPF_IVM[row.se_seismo], axis=1)

In [6]:
# replace impf 3 --> 5 for 2-storeys and more
gdf_bem_subcomps.loc[((gdf_bem_subcomps.bd_3_floor+gdf_bem_subcomps.bd_2_floor)>0.5)
                     &(gdf_bem_subcomps.impf_FL==3), "impf_FL"] = 5

In [7]:
# replace impf 4 --> 6 for 2-storeys and more
gdf_bem_subcomps.loc[((gdf_bem_subcomps.bd_3_floor+gdf_bem_subcomps.bd_2_floor)>0.5)
                     &(gdf_bem_subcomps.impf_FL==4), "impf_FL"] = 6

In [8]:
# remove for now unnecessary cols and prepare gdf for CLIMADA Exposure
gdf_bem_subcomps.rename({'valhum' : 'value'}, axis=1)
for col in ['iso3', 'sector', 'valfis', 'se_seismo']:
    gdf_bem_subcomps.pop(col)

In [9]:
gdf_bem_subcomps.valhum.sum()

39170907.24501941

In [10]:
exp = Exposures(gdf_bem_subcomps)
exp.gdf.rename({'valhum': 'value'}, axis=1, inplace=True)
exp.value_unit = 'Pop. count'
exp.gdf['longitude'] = exp.gdf.geometry.x
exp.gdf['latitude'] = exp.gdf.geometry.y
exp.gdf = exp.gdf[~np.isnan(
    exp.gdf.latitude)]  # drop nan centroids
exp.gdf.head()

,id_1x,cpx,value,bd_1_floor,bd_2_floor,bd_3_floor,geometry,impf_FL,longitude,latitude
56,157350270,3,3.203340,66.665,33.335,0.0,POINT (118.45417 24.47083),6,118.454167,24.470833
57,157350270,3,4.835457,66.665,33.335,0.0,POINT (118.45417 24.47083),6,118.454167,24.470833
58,157350270,3,5.618275,66.665,33.335,0.0,POINT (118.45417 24.47083),6,118.454167,24.470833
80,157350270,3,4.031061,66.665,33.335,0.0,POINT (118.45417 24.47083),6,118.454167,24.470833
81,157350270,3,6.084906,66.665,33.335,0.0,POINT (118.45417 24.47083),6,118.454167,24.470833


#### Get lat/lon min/max from exposure

In [11]:
lat_min, lat_max, lon_min, lon_max = exp.gdf['latitude'].min(), exp.gdf['latitude'].max(), exp.gdf['longitude'].min(), exp.gdf['longitude'].max()

In [12]:
lat_min, lat_max, lon_min, lon_max

(21.912500005759863, 25.27916667242634, 118.28749999999884, 122.00416666666847)

## Load hazard maps for all tiles covering the lat/lon extent of exposure

In [13]:
from climada.util.constants import SYSTEM_DIR
hazard_dir = SYSTEM_DIR/"hazard"/"coastal_flood"/"venDEM_scaled_1km"

In [14]:
# get tiles covering the exposure extent
tiles = coastal_flood_hazard.find_tiles(lat_min, lat_max, lon_min, lon_max)

In [15]:
tiles

['N21E118',
 'N21E119',
 'N21E120',
 'N21E121',
 'N21E122',
 'N22E118',
 'N22E119',
 'N22E120',
 'N22E121',
 'N22E122',
 'N23E118',
 'N23E119',
 'N23E120',
 'N23E121',
 'N23E122',
 'N24E118',
 'N24E119',
 'N24E120',
 'N24E121',
 'N24E122',
 'N25E118',
 'N25E119',
 'N25E120',
 'N25E121',
 'N25E122']

In [16]:
CF_haz_2020 = coastal_flood_hazard.generate_hazard_object(tiles, hazard_dir, 'RCP45', '2020', 'FL')

Tile directory /Users/simonameiler/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N21E118/RCP45_2020 not found, skipping...
Tile directory /Users/simonameiler/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N21E119/RCP45_2020 not found, skipping...
Tile directory /Users/simonameiler/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N22E118/RCP45_2020 not found, skipping...
Tile directory /Users/simonameiler/climada/data/hazard/coastal_flood/venDEM_scaled_1km/N22E122/RCP45_2020 not found, skipping...


ValueError: Cannot determine common CRS for concatenation inputs, got ['WGS 84', 'WGS 84']. Use `to_crs()` to transform geometries to the same CRS before merging.

In [ ]:
CF_haz_2020.intensity

In [ ]:
CF_haz_2020.centroids.plot()

In [ ]:
CF_haz_2020.plot_intensity(event=0)

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': ccrs.PlateCarree()})
CF_haz_2020.plot_intensity(event=0, axis=ax)

plt.show()


In [ ]:
CF_haz_2020.intensity.max(axis=1).data

In [ ]:
CF_haz_2050 = coastal_flood_hazard.generate_hazard_object(tiles, hazard_dir, 'RCP45', '2050', 'FL')

In [ ]:
CF_haz_2050.plot_intensity(event=0)

In [ ]:
CF_haz_2100 = coastal_flood_hazard.generate_hazard_object(tiles, hazard_dir, 'RCP45', '2100', 'FL')

In [ ]:
CF_haz_2100.plot_intensity(event=0)

In [ ]:
CF_haz_2050.intensity

In [ ]:
CF_haz_2020.frequency.size

In [ ]:
CF_haz_2020.event_id

### Question:
Is it a problem that event_id and event_name are "sanitized" and that intensity crs matrices are stacked vertically during the concat/append routine when calling `generate_hazard_object`?

## Impact functions

### Option 1 - merging damage threshold into impact function

In [ ]:
impf_set_fl = vulnerability.IMPF_SET_FL_IVM

In [ ]:
from climada.entity import ImpactFunc, ImpactFuncSet
impf_set_ivm_step = ImpactFuncSet()

In [ ]:
# The threshold of building damage after which all people are displaced. Below, no-one is displaced.
building_thes = 0.55 # 55% iDMC v1; CIMA: 30% for Somalia to 60% for other countries. 

for imp_id in impf_set_fl.get_ids(haz_type='FL'):
    impf_set_fl.get_func(fun_id=imp_id)
    y = impf_set_fl.get_func(fun_id=imp_id)[0].intensity
    x = impf_set_fl.get_func(fun_id=imp_id)[0].mdd
    flood_thres = np.interp(building_thes, x, y)
    print('ID: '+str(imp_id)+' - threshold stepfunction: '+str(flood_thres))
    impf_set_ivm_step.append(
                ImpactFunc.from_step_impf(
                    intensity=(0,  flood_thres, flood_thres *10),
                    haz_type='FL',
                    impf_id=imp_id,
                    intensity_unit = 'm'
                )
    )

### Impacts

In [ ]:
from climada.engine import ImpactCalc

impcalc = ImpactCalc(exp, impf_set_ivm_step, CF_haz_2020)
impact = impcalc.impact()

In [ ]:
print('Annual average displacement: ' + "{:,.0f}".format(impact.aai_agg))

In [ ]:
freqd_curve = impact.calc_freq_curve()

In [ ]:
freqd_curve.plot()

In [ ]:
impact.plot_hexbin_eai_exposure(pop_name=False, ignore_zero=True)

### Option 2 - postprocessing results by damage threshold

In [ ]:
from climada.engine import ImpactCalc

exp.gdf.rename({'value' : 'valhum'}, axis=1, inplace=True)
exp.gdf['value'] = 1
exp.gdf['impf_FL'] = exp.gdf['se_seismo'].map(vulnerability.DICT_PAGER_FLIMPF_CIMA) 

In [ ]:
# compute damage fractions on buildings; keep "events", i.e. in case of flood RPs individually
impact_bldg = ImpactCalc(exp, vulnerability.IMPF_SET_FL_CIMA, CF_haz_2100).impact(save_mat=True)

In [ ]:
# set displacement-damage threshold(s)
dmg_thresh_low = 0.3
dmg_thresh_med = 0.45
dmg_thresh_high = 0.6

In [ ]:
# decide on whether threshold reached
displ_low = int(impact_bldg.imp_mat > dmg_thresh_low)
displ_med = int(impact_bldg.imp_mat > dmg_thresh_med)
displ_high = int(impact_bldg.imp_mat > dmg_thresh_high)

In [ ]:
# compute displacement impacts (exemplified only for median threshold now); store with gdf. 
# Problem: this likely blows up memory as 0s are now explicit
exp.gdf[['imp_rp1','imp_rp10','imp_rp25','imp_rp100','imp_rp250','imp_rp1000']] = exp.gdf.valhum*displ_med.data

In [ ]:
# explicitly compute AED agg. for median threshold scenario (others analogous)
exp.gdf['aed_med'] = exp.gdf['imp_rp1'] *1 + exp.gdf['imp_rp10'] *1/10 + exp.gdf['imp_rp25']*1/25 + exp.gdf['imp_rp100']*1/100 + exp.gdf['imp_rp250'] *1/250 + exp.gdf['imp_rp1000'] *1/1000

In [ ]:
# aggregate impacts by admin-1
exp.gdf['admin1'] = exposure.assign_admin1_attr(exp.gdf.copy(), exposure.path_admin1_attrs, source='gadm')
gdf_admin1_imps = exp.gdf.groupby('admin1')[['imp_rp1','imp_rp10','imp_rp25','imp_rp100','imp_rp250','imp_rp1000', 'aed_med']].sum()

In [ ]:
# aggregate over whole exposure